In [ ]:
import random
import torch
import pandas as pd
import numpy as np

device="cuda" if torch.cuda.is_available() else "cpu"


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd /content/drive/My Drive/

/content/drive/My Drive


In [ ]:
#converting embeddings column from string to numpy array
text_chunks_and_embeddings_df=pd.read_csv("text_chunks_and_embeddings_df.csv")
text_chunks_and_embeddings_df["embedding"]=text_chunks_and_embeddings_df["embedding"].apply(lambda x:np.fromstring(x.strip("[]"),sep=" "))

#converting dataframe into dictionaries
pages_and_chunks=text_chunks_and_embeddings_df.to_dict(orient="records")

#converting embeddings to tensor
embeddings=torch.tensor(np.array(text_chunks_and_embeddings_df["embedding"].tolist()),dtype=torch.float32).to(device)
embeddings.shape

torch.Size([1680, 768])

In [ ]:
text_chunks_and_embeddings_df.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,-39,Human Nutrition: 2020 Edition UNIVERSITY OF HA...,308,42,77.00,"[0.067424275, 0.0902281851, -0.00509549817, -0..."
1,-38,Human Nutrition: 2020 Edition by University of...,210,30,52.50,"[0.0552156083, 0.0592139438, -0.0166167226, -0..."
2,-37,Contents Preface University of Hawai‘i at Māno...,766,114,191.50,"[0.0279801711, 0.0339814313, -0.0206426643, 0...."
3,-36,Lifestyles and Nutrition University of Hawai‘i...,941,142,235.25,"[0.0682566911, 0.0381274782, -0.00846855436, -..."
4,-35,The Cardiovascular System University of Hawai‘...,998,152,249.50,"[0.0330264196, -0.00849765539, 0.00957158115, ..."


In [ ]:
pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.9 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import util,SentenceTransformer

In [ ]:
embedding_model=SentenceTransformer(model_name_or_path="all-mpnet-base-v2",device=device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
query="macronutrients functions"
print(f"Query:{query}")

query_embedding=embedding_model.encode(query,convert_to_tensor=True)

from time import perf_counter as timer

start_time=timer()
dot_scores=util.dot_score(a=query_embedding,b=embeddings)[0]
end_time=timer()

print(f"TIme took to get scores on {len(embeddings)} embeddings:{end_time-start_time:.5f} seconds")

top_results_dot_product=torch.topk(dot_scores,k=5)
top_results_dot_product



Query:macronutrients functions
TIme took to get scores on 1680 embeddings:0.02131 seconds


torch.return_types.topk(
values=tensor([0.6926, 0.6738, 0.6646, 0.6536, 0.6473], device='cuda:0'),
indices=tensor([42, 47, 41, 51, 46], device='cuda:0'))

In [ ]:
import textwrap

def print_wrapped(text,wrap_length=80):
  wrapped_text=textwrap.fill(text,wrap_length)
  print(wrapped_text)

In [ ]:
print(f"Query:`{query}`\n")
print("Results:")

for score,idx in zip(top_results_dot_product[0],top_results_dot_product[1]):
  print(f"Score: {score:.4f}")

  print("Text:")

  print_wrapped(pages_and_chunks[idx]["sentence_chunk"])

  print(f"Page Number:{pages_and_chunks[idx]['page_number']}")
  print("\n")

Query:`macronutrients functions`

Results:
Score: 0.6926
Text:
Macronutrients Nutrients that are needed in large amounts are called
macronutrients. There are three classes of macronutrients: carbohydrates,
lipids, and proteins. These can be metabolically processed into cellular energy.
The energy from macronutrients comes from their chemical bonds. This chemical
energy is converted into cellular energy that is then utilized to perform work,
allowing our bodies to conduct their basic functions. A unit of measurement of
food energy is the calorie. On nutrition food labels the amount given for
“calories” is actually equivalent to each calorie multiplied by one thousand. A
kilocalorie (one thousand calories, denoted with a small “c”) is synonymous with
the “Calorie” (with a capital “C”) on nutrition food labels. Water is also a
macronutrient in the sense that you require a large amount of it, but unlike the
other macronutrients, it does not yield calories. Carbohydrates Carbohydrates
are m

### Making Function for Semantic Searching

In [ ]:
def retrieve_relevant_resources(query:str,embeddings:torch.tensor,
                                model:SentenceTransformer=embedding_model,
                                n_resources_to_return:int=5,
                                print_time:bool=True):

  query_embedding=model.encode(query,convert_to_tensor=True)

  dot_scores=util.dot_score(query_embedding,embeddings)[0]
  #can use cosine_similarity here

  scores,indices=torch.topk(input=dot_scores,k=n_resources_to_return)

  return scores,indices


def print_top_results_and_scores(query:str,embeddings:torch.tensor,
                                 pages_chunks:list[dict]=pages_and_chunks,
                                 n_resources_to_return: int=5):

  scores,indices=retrieve_relevant_resources(query=query,
                                             embeddings=embeddings,
                                             n_resources_to_return=n_resources_to_return)
  print(f"Query :`{query}`\n")
  print("Results:")

  for score,index in zip(scores,indices):
    print(f"Score: {score:.4f}")

    print_wrapped(pages_and_chunks[index]["sentence_chunk"])

    print(f"Page Number: {pages_and_chunks[index]['page_number']}")

    print("\n")


Faiss Can BE IMPLEMENTED FOR VECTOR COMPARISION Incase of Large Vector Space

In [ ]:
gpu_memory_bytes=torch.cuda.get_device_properties(0).total_memory
gpu_memory_gb=round(gpu_memory_bytes/(2**30))
print(f"Available GPU memory:{gpu_memory_gb} GB")

Available GPU memory:15 GB


In [ ]:
# Note: the following is Gemma focused, however, there are more and more LLMs of the 2B and 7B size appearing for local use.
if gpu_memory_gb < 5.1:
    print(f"Your available GPU memory is {gpu_memory_gb}GB, you may not have enough memory to run a Gemma LLM locally without quantization.")
elif gpu_memory_gb < 8.1:
    print(f"GPU memory: {gpu_memory_gb} | Recommended model: Gemma 2B in 4-bit precision.")
    use_quantization_config = True
    model_id = "google/gemma-2b-it"
elif gpu_memory_gb < 19.0:
    print(f"GPU memory: {gpu_memory_gb} | Recommended model: Gemma 2B in float16 or Gemma 7B in 4-bit precision.")
    use_quantization_config = False
    model_id = "google/gemma-2b-it"
elif gpu_memory_gb > 19.0:
    print(f"GPU memory: {gpu_memory_gb} | Recommend model: Gemma 7B in 4-bit or float16 precision.")
    use_quantization_config = False
    model_id = "google/gemma-7b-it"

print(f"use_quantization_config set to: {use_quantization_config}")
print(f"model_id set to: {model_id}")

GPU memory: 15 | Recommended model: Gemma 2B in float16 or Gemma 7B in 4-bit precision.
use_quantization_config set to: False
model_id set to: google/gemma-2b-it


In [ ]:
pip install bitsandbytes

In [ ]:
access_token="#####"

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig

quantization_config=BitsAndBytesConfig(load_in_4bit=True,
                                       bnb_4bit_compute_dtype=torch.float16)

attn_implementation="sdpa"

model_id=model_id


tokenizer=AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_id,token=access_token)

llm_model=AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_id,
                                               torch_dtype=torch.float16,
                                               quantization_config=quantization_config if use_quantization_config else None,
                                               low_cpu_mem_usage=False,
                                               attn_implementation=attn_implementation,token=access_token)

if not use_quantization_config:
  llm_model.to("cuda")



config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
llm_model

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
      )
    )
    (norm): GemmaR

In [ ]:
def get_model_num_params(model:torch.nn.Module):
  return sum([param.numel() for param in model.parameters()])

get_model_num_params(llm_model)

2506172416

In [ ]:
gpt4_questions = [
    "What are the macronutrients, and what roles do they play in the human body?",
    "How do vitamins and minerals differ in their roles and importance for health?",
    "Describe the process of digestion and absorption of nutrients in the human body.",
    "What role does fibre play in digestion? Name five fibre containing foods.",
    "Explain the concept of energy balance and its importance in weight management."
]


manual_questions = [
    "How often should infants be breastfed?",
    "What are symptoms of pellagra?",
    "How does saliva help with digestion?",
    "What is the RDI for protein per day?",
    "water soluble vitamins"
]

query_list = gpt4_questions + manual_questions

In [ ]:
def prompt_formatter(query: str,
                     context_items: list[dict]) -> str:

    context = "- " + "\n- ".join([item["sentence_chunk"] for item in context_items])

    base_prompt = """Based on the following context items, please answer the query.
Give yourself room to think by extracting relevant passages from the context before answering the query.
Don't return the thinking, only return the answer.
Make sure your answers are as explanatory as possible.
Use the following examples as reference for the ideal answer style.
\nExample 1:
Query: What are the fat-soluble vitamins?
Answer: The fat-soluble vitamins include Vitamin A, Vitamin D, Vitamin E, and Vitamin K. These vitamins are absorbed along with fats in the diet and can be stored in the body's fatty tissue and liver for later use. Vitamin A is important for vision, immune function, and skin health. Vitamin D plays a critical role in calcium absorption and bone health. Vitamin E acts as an antioxidant, protecting cells from damage. Vitamin K is essential for blood clotting and bone metabolism.
\nExample 2:
Query: What are the causes of type 2 diabetes?
Answer: Type 2 diabetes is often associated with overnutrition, particularly the overconsumption of calories leading to obesity. Factors include a diet high in refined sugars and saturated fats, which can lead to insulin resistance, a condition where the body's cells do not respond effectively to insulin. Over time, the pancreas cannot produce enough insulin to manage blood sugar levels, resulting in type 2 diabetes. Additionally, excessive caloric intake without sufficient physical activity exacerbates the risk by promoting weight gain and fat accumulation, particularly around the abdomen, further contributing to insulin resistance.
\nExample 3:
Query: What is the importance of hydration for physical performance?
Answer: Hydration is crucial for physical performance because water plays key roles in maintaining blood volume, regulating body temperature, and ensuring the transport of nutrients and oxygen to cells. Adequate hydration is essential for optimal muscle function, endurance, and recovery. Dehydration can lead to decreased performance, fatigue, and increased risk of heat-related illnesses, such as heat stroke. Drinking sufficient water before, during, and after exercise helps ensure peak physical performance and recovery.
\nNow use the following context items to answer the user query:
{context}
\nRelevant passages: <extract relevant passages from the context here>
User query: {query}
Answer:"""


    base_prompt = base_prompt.format(context=context, query=query)


    dialogue_template = [
        {"role": "user",
        "content": base_prompt}
    ]


    prompt = tokenizer.apply_chat_template(conversation=dialogue_template,
                                          tokenize=False,
                                          add_generation_prompt=True)
    return prompt

In [ ]:
def ask(query,
        temperature=0.7,
        max_new_tokens=512,
        format_answer_text=True,
        return_answer_only=True):
    """
    Takes a query, finds relevant resources/context and generates an answer to the query based on the relevant resources.
    """

    # Get just the scores and indices of top related results
    scores, indices = retrieve_relevant_resources(query=query,
                                                  embeddings=embeddings)

    # Create a list of context items
    context_items = [pages_and_chunks[i] for i in indices]

    # Add score to context item
    for i, item in enumerate(context_items):
        item["score"] = scores[i].cpu() # return score back to CPU

    # Format the prompt with context items
    prompt = prompt_formatter(query=query,
                              context_items=context_items)

    # Tokenize the prompt
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate an output of tokens
    outputs = llm_model.generate(**input_ids,
                                 temperature=temperature,
                                 do_sample=True,
                                 max_new_tokens=max_new_tokens)

    # Turn the output tokens into text
    output_text = tokenizer.decode(outputs[0])

    if format_answer_text:
        # Replace special tokens and unnecessary help message
        output_text = output_text.replace(prompt, "").replace("<bos>", "").replace("<eos>", "").replace("Sure, here is the answer to the user query:\n\n", "")

    # Only return the answer without the context items
    if return_answer_only:
        return output_text

    return output_text, context_items

In [ ]:
query = random.choice(query_list)
print(f"Query: {query}")

# Answer query with context and return context
answer, context_items = ask(query=query,
                            temperature=0.7,
                            max_new_tokens=512,
                            return_answer_only=False)

print(f"Answer:\n")
print_wrapped(answer)
# print(f"Context items:")
# context_items

Query: What role does fibre play in digestion? Name five fibre containing foods.
Answer:

Sure, here are the relevant passages from the context that answer the user's
query:  **Passage 1:** >"The breakdown of fiber in the large intestine releases
short-chain fatty acids. These molecules have been found to nourish colonic
cells, inhibit colonic inflammation, and stimulate the immune system (thereby
providing protection of the colon from harmful substances). Additionally, the
bacterial indigestible fiber, mostly insoluble, increases stool bulk and
softness increasing transit time in the large intestine and facilitating feces
elimination. One phenomenon of consuming foods high in fiber is increased gas,
since the byproducts of bacterial digestion of fiber are gases."  **Passage 2:**
>"Dietary fiber is categorized as either water-soluble or insoluble. Some
examples of soluble fibers are inulin, pectin, and guar gum and they are found
in peas, beans, oats, barley, and rye. Cellulose and lig